In [ ]:
#This code prepares input data for training


In [1]:
import pandas as pd
import numpy as np
import itertools    
import random    
import math
import gzip

import torch
import torch.nn as nn


import matplotlib.pyplot as plt 
from Bio.Seq import Seq #biopython
import subprocess
from pathlib import Path


In [ ]:
#########################
######################### Produce data from centromere and "chromatin proper"
#########################

windowsize=50


from Bio import SeqIO

num_random_read_in_seq = 50000

def create_data_for_read(oneseq, onestart):
    oneseq = oneseq.upper()
    #print(use_i)

    onepos = [0]*onestart + list(range(0,len(oneseq)-onestart))

    def getavglist(lst):
        return sum(lst)/len(lst)


    focuslen = len(oneseq)

    list_randomstart = [random.randint(0,focuslen-windowsize) for x in range(0,num_random_read_in_seq)]
    list_randomseq = [oneseq[randomstart:(randomstart+windowsize)] for randomstart in list_randomstart]
    list_randompos = [getavglist(onepos[randomstart:(randomstart+windowsize)]) for randomstart in list_randomstart]

    mer8 = pd.DataFrame({
        "seq":list_randomseq,
        "score":list_randompos
    })
    return(mer8)

def get_reads_from_record(record):
    print(record.id)
    
    if len(record.seq)<100000:
        return(pd.DataFrame({
            "start":[],
            "seq":[]
        }))
    else:
        return(create_data_for_read(str(record.seq), 0))

all_nc = pd.concat([
    get_reads_from_record(record) for record
    in SeqIO.parse("/corgi/otherdataset/t2t/other/simplified_noncentromere.fa", "fasta")])

all_c = pd.concat([
    get_reads_from_record(record) for record
    in SeqIO.parse("/corgi/otherdataset/t2t/other/simplified_centromere.fa", "fasta")])

all_nc["score"] = 0 
all_c["score"] = 1  
print("done")

all_c.to_pickle("/husky/otherdataset/karimian2024/t2t_centro_50.pickle")
all_nc.to_pickle("/husky/otherdataset/karimian2024/t2t_noncentro_50.pickle")





In [12]:
#########################
######################### Produce data from long read of telomeres
#########################


def findall(p, s):
    '''Yields all the positions of
    the pattern p in the string s.'''
    i = s.find(p)
    while i != -1:
        yield i
        i = s.find(p, i+1)


#########################
# One issue here is that we could match a bit more; something for a regexp?
def find_telomere_length(line):
    i = 0
    while True:
        ne = line.find("TAACCC",i)
        if ne == -1:
            break
        if ne-i > 18: #seems forgiving enough
            break
        i = ne+6
    return(i)


def read_longread_file(fname):
    mycmd=subprocess.getoutput('zcat '+fname+' | grep CCCTCCGATA')

    def remove_telotag(line):
        return line[0:line.rfind('CCCTCCGATA')]

    lines=mycmd.splitlines()
    lines = [remove_telotag(s) for s in lines]

    #Reverse complement all lines such that they begin with the telomere
    lines = [str(Seq(s).reverse_complement()) for s in lines]

    lens = [find_telomere_length(s) for s in lines]
    df = pd.DataFrame({
        "start":lens,
        "seq":lines
    })
    df = df[df["start"]>50]

    return(df)



def flatten(xss):
    return [x for xs in xss for x in xs]

### random number on 0... maxval 
def randint_logscale(maxval):
    r=random.uniform(0,math.log(1+maxval))
    return(int(math.exp(r)-1))    


def create_data_for_read(seqdata, use_i):
    #print(use_i)

    oneseq=seqdata["seq"][use_i].upper()
    onestart=seqdata["start"][use_i]

    onepos = [0]*onestart + list(range(0,len(oneseq)-onestart))

    mer8 = pd.DataFrame({
        "seq":[oneseq],
        "pos":[onepos]
    })
    return(mer8)

def create_data_for_file(fname):
    print(fname)
    seqdata = read_longread_file(str(fname))
    list_for_file = [create_data_for_read(seqdata, use_i) for use_i in seqdata.index]
    return(pd.concat(list_for_file))


#seqdata = read_longread_file("/husky/otherdataset/karimian2024/more/3048691")
directory = "/husky/otherdataset/karimian2024/more"
files = Path(directory).glob('*')
list_mer8 = [create_data_for_file(fname) for fname in files]
mer8 = pd.concat(list_mer8)

mer8[1:10000].to_pickle("/husky/otherdataset/karimian2024/train_telo_xbp.pickle")

/husky/otherdataset/karimian2024/more/3048693
/husky/otherdataset/karimian2024/more/3048691
/husky/otherdataset/karimian2024/more/3048695
/husky/otherdataset/karimian2024/more/3048694
/husky/otherdataset/karimian2024/more/3048696
/husky/otherdataset/karimian2024/more/3048698
/husky/otherdataset/karimian2024/more/3048697
/husky/otherdataset/karimian2024/more/3048699
/husky/otherdataset/karimian2024/more/3048700
/husky/otherdataset/karimian2024/more/3048701
/husky/otherdataset/karimian2024/more/3048703
/husky/otherdataset/karimian2024/more/3048702
/husky/otherdataset/karimian2024/more/3048709
/husky/otherdataset/karimian2024/more/3048704
/husky/otherdataset/karimian2024/more/3048705
/husky/otherdataset/karimian2024/more/3048708
/husky/otherdataset/karimian2024/more/3048706
/husky/otherdataset/karimian2024/more/3048707
